# Библиотеки 

In [1]:
import pandas as pd
import os
import numpy as np

# Файлы

In [2]:
print('Dataset:')
for dirname, _, filenames in os.walk('input'): # Os_walk генерирует именафайлов в дереве каталогов
    for filename in filenames:
        print(os.path.join(dirname, filename))

Dataset:
input\interactions.csv
input\items.csv
input\items_preprocessed.csv
input\users.csv
input\users_preprocessed.csv
input\~$teractions_preprocessed.pickle
input\.ipynb_checkpoints\interactions-checkpoint.csv
input\.ipynb_checkpoints\items-checkpoint.csv
input\.ipynb_checkpoints\items_preprocessed-checkpoint.csv
input\.ipynb_checkpoints\users-checkpoint.csv
input\.ipynb_checkpoints\users_preprocessed-checkpoint.csv


# interactions

In [100]:
data = pd.read_csv('input/interactions.csv')
data
data['start_date'] = pd.to_datetime(data['start_date'])

### Проверим данные на дубликаты

In [101]:
len(data['user_id'].unique())

158325

In [102]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562617 entries, 0 to 1562616
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   user_id     1562617 non-null  int64         
 1   item_id     1562617 non-null  int64         
 2   progress    1562617 non-null  int64         
 3   rating      323571 non-null   float64       
 4   start_date  1562617 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 59.6 MB


In [103]:
duplicates = data.duplicated(subset=['user_id', 'item_id'], keep=False)
duplicates.sum() # Целых 224 дубликата

224

In [104]:
data_duplicates = data[duplicates].sort_values(by=['user_id', 'start_date'])
data = data[~duplicates] # Знак ~ заменяет not

In [105]:
data_duplicates

,user_id,item_id,progress,rating,start_date
607888,703,44081,64,NaN,2018-10-14
1046199,703,44081,100,NaN,2019-05-05
351581,1757,279535,3,0.0,2018-06-16
1337090,1757,279535,1,NaN,2019-09-17
342919,1889,4292,40,0.0,2018-06-12
...,...,...,...,...,...
1286478,163719,27040,7,NaN,2019-08-24
467200,165754,303933,9,NaN,2018-08-09
1495675,165754,303933,42,0.0,2019-11-30
601924,165939,16044,98,NaN,2018-10-11


#### Емае, зачеееем, зачем нам sql, если тут есть даже агрегация по групам

In [106]:
data_duplicates = data_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})
data_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 112 entries, (703, 44081) to (165939, 16044)
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   progress    112 non-null    int64         
 1   rating      51 non-null     float64       
 2   start_date  112 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 4.7 KB


In [107]:
data = data.append(data_duplicates.reset_index(), ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562505 entries, 0 to 1562504
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   user_id     1562505 non-null  int64         
 1   item_id     1562505 non-null  int64         
 2   progress    1562505 non-null  int64         
 3   rating      323563 non-null   float64       
 4   start_date  1562505 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 59.6 MB


### Наш столбец без дубликатов

In [108]:
data

,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01
2,80061,26540,69,4.0,2018-01-01
3,12811,301895,16,NaN,2018-01-01
4,5778,127872,100,NaN,2018-01-01
...,...,...,...,...,...
1562500,160045,291585,85,NaN,2018-02-25
1562501,161224,7819,58,NaN,2018-04-25
1562502,163719,27040,99,NaN,2018-05-11
1562503,165754,303933,42,0.0,2018-08-09


### Далее немного меняем типы данных, чтобы использовалось меньше памяти

In [109]:
data.nunique()


user_id       158325
item_id        63758
progress         102
rating             6
start_date       730
dtype: int64

In [110]:
data['progress'] = data['progress'].astype(np.int8)
data['rating'] = data['rating'].astype(pd.SparseDtype(np.float32, np.nan))
data

,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01
2,80061,26540,69,4.0,2018-01-01
3,12811,301895,16,NaN,2018-01-01
4,5778,127872,100,NaN,2018-01-01
...,...,...,...,...,...
1562500,160045,291585,85,NaN,2018-02-25
1562501,161224,7819,58,NaN,2018-04-25
1562502,163719,27040,99,NaN,2018-05-11
1562503,165754,303933,42,0.0,2018-08-09


In [111]:
# Записываем новую таблицу в файл, не знаю почему, .pickle жаловался на utf-8, поэтому сохраню в csv

data.to_csv('input/interactions_preprocessed.csv', index=False)

## Users

In [61]:
data2 = pd.read_csv('input/users.csv')
data2

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0
...,...,...,...
137249,166060,25_34,0.0
137250,166061,35_44,1.0
137251,166062,45_54,1.0
137252,166065,25_34,0.0


In [16]:
data2.nunique()

user_id    137254
age             6
sex             2
dtype: int64

### Меняем типы данных

In [48]:
data2['age'] = data2['age'].astype('category')
data2['sex'] = data2['sex'].astype(pd.SparseDtype(np.float32, np.nan))
data2

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0
...,...,...,...
137249,166060,25_34,0.0
137250,166061,35_44,1.0
137251,166062,45_54,1.0
137252,166065,25_34,0.0


In [18]:
interaction = data['user_id'].unique()

# np.intersect1d находит элементы которые принадлежат обоим массивам

common = len(np.intersect1d(interaction, data2['user_id']))

# np.setdiff1d находит элементы, которые есть в первом массиве, но нет во втором

users_only_in_interaction = len(np.setdiff1d(interaction, data2['user_id']))
users_only_features = len(np.setdiff1d(data2['user_id'], interaction))

total = common + users_only_in_interaction + users_only_features

print(f'Кол-во пользователей - {total}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common} ({common / total * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total * 100:.2f}%)')


Кол-во пользователей - 164771
Кол-во пользователей c взаимодействиями и фичами - 130808 (79.39%)
Кол-во пользователей только c взаимодействиями - 27517 (16.70%)
Кол-во пользователей только c фичами - 6446 (3.91%)


In [19]:
data2.to_csv('input/users_preprocessed.csv', index=False)

# Items

In [20]:
data_items = pd.read_csv('input/items.csv')
data_items

,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873
...,...,...,...,...,...
63753,143203,Игрушка палача,"Эротические романы,Любовно-фантастические рома...",Эрика Адамс,2019
63754,27324,Любовь под развесистой клюквой,Современные любовные романы,Маргарита Южина,NaN
63755,359711,"Куда уходят грешницы, или Гробница Наполеона",Современные детективы,Наталья Андреева,NaN
63756,283955,Сварить медведя,"Зарубежные детективы,Исторические детективы,За...",Микаель Ниеми,2017


In [36]:
new_row = {'id': 10309, 'title': None, 'genres': None, 'authors': None, 'year': None}
data_items.append(new_row,ignore_index=True)

In [38]:
# Ф-я считает количество занимаемой памяти
def num_bytes_format(num_bytes, float_prec=4):
    units = ['bytes', 'Kb', 'Mb', 'Gb', 'Tb', 'Pb', 'Eb']
    for unit in units[:-1]:
        if abs(num_bytes) < 1000:
            return f'{num_bytes:.{float_prec}f} {unit}'
        num_bytes /= 1000
    return f'{num_bytes:.4f} {units[-1]}'

In [41]:
num_bytes = data_items.memory_usage(deep=True).sum()
num_bytes_format(num_bytes)

'33.5359 Mb'

In [42]:
data_items.nunique()

id         63759
title      58093
genres     11091
authors    17024
year        1074
dtype: int64

### Переводим в категориальный тип

In [43]:
for col in ['genres', 'authors', 'year']:
    data_items[col] = data_items_new[col].astype('category')

In [44]:
data_items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63759 entries, 0 to 63758
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id       63759 non-null  float64 
 1   title    63758 non-null  object  
 2   genres   63753 non-null  category
 3   authors  56700 non-null  category
 4   year     49508 non-null  category
dtypes: category(3), float64(1), object(1)
memory usage: 2.5+ MB


In [45]:
num_bytes = data_items.memory_usage(deep=True).sum()
num_bytes_format(num_bytes) # Занимаемая память уменьшилаь почти в 2 раза

'18.6995 Mb'

In [46]:
interaction_items = data['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, data_items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, data_items['id']))
items_only_features = len(np.setdiff1d(data_items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 63759
Кол-во книг c взаимодействиями и фичами - 63758 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 1 (0.00%)


In [47]:
data_items_new.to_csv('input/items_preprocessed.csv', index=False)